In [1]:
from langchain_ollama import ChatOllama,OllamaEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,PromptTemplate
from langchain_core.messages import HumanMessage,AIMessage,SystemMessage
from langchain.vectorstores import FAISS
from langchain_chroma import Chroma
from langchain.document_loaders import PyPDFLoader,Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough,RunnableLambda,RunnableParallel,RunnableSequence
from langchain_core.output_parsers import StrOutputParser

# 1. Define Chat model and Embedding model

In [3]:
llm=ChatOllama(
    model="gemma3:1b",
    temperature=.4
)
ollama_embeddings = OllamaEmbeddings(model="mxbai-embed-large:latest")

huggingface_embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/e5-base-v2"
)

parser=StrOutputParser()

# 2. Define the document loaders and folder loaders

In [6]:
import os

def document_loader(file_path):
    if file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    elif file_path.endswith(".docx"):
        loader = Docx2txtLoader(file_path)
    else:
        raise ValueError("Unsupported file format")
    
    documents=loader.load()
    for doc in documents:
        doc.page_content = " ".join(doc.page_content.split())
    return documents

def folder_loader(folder_path:str):
    documents = []
    full_folder_path=os.path.abspath(folder_path)
    for filename in os.listdir(full_folder_path):
        file_path = os.path.join(full_folder_path, filename)
        if os.path.isfile(file_path):
            with open("embedded_files.txt", "a") as f:
                f.write(f"{file_path}\n")
            documents.extend(document_loader(file_path))
    return documents

    
# len(folder_loader("Files/CCE"))

# 3. Define the vector store and text splitters

In [ ]:
vector_store = Chroma(
    embedding_function=huggingface_embeddings,
    persist_directory="vector_store/chroma_vector_store",
    collection_name="user2_collection"
)


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=900,
    chunk_overlap=200
)

# text_splitter.split_documents(folder_loader("Files/EEE"))

In [7]:
splitted_text = text_splitter.split_documents(folder_loader("Files/CCE"))
splitted_text+= text_splitter.split_documents(folder_loader("Files/EEE"))
vector_data=vector_store.add_documents(splitted_text)

### Load the local vector store

In [ ]:
persist_directory = "vector_store/chroma_vector_store" 
# Load the Chroma vector store from the local directory
vector_store = Chroma(
    persist_directory=persist_directory,
    embedding_function=huggingface_embeddings,
    collection_name="user2_collection"
)

In [60]:
vector_store.get(include=["embeddings","metadatas","documents"])

{'ids': ['b7df3ba8-61fb-409c-8419-17cd8cde1b7d',
  '7f8e1052-fbd8-4cee-8b62-6d2d056e45e6',
  '29c8230e-d0c6-4a39-a073-f61ea7d7a9ec',
  'e5b0ee6a-ca48-411f-a5ed-cf80c382cc20',
  '76a62dee-8b29-46c3-bd9b-7559b4f4581b',
  '8157b911-02d2-4c3a-acf5-ee1cd08f18cd',
  'd1a46c0f-d283-4622-97a7-869481d8ba44',
  '28d800ea-8c53-46ee-aa73-53cccbdf2997',
  '9af8f896-b755-42c6-a6aa-889cf3372c36',
  'fa5c3db4-521b-4a94-b98f-514097453ca0',
  'a9e59247-065f-4c3e-9a4f-eeea8e888cef',
  'ddff35f1-7666-4651-be21-b6ab85a3454b',
  '1e48a431-8b93-4e42-a1f6-baede6691269',
  'bec0059a-4bcc-462d-8a8a-0e413a9fe10e',
  '6a327da5-73cf-427f-b8d8-f6ad81b8aca0',
  '7d4d96ac-326d-4e79-88b9-cee1854c194a',
  '25661291-f10e-4003-bd3f-86ac658451ba',
  '95e17b92-69fe-4f35-8759-d19e87ae5732',
  '338c7890-7974-42e6-9100-16678a605eb0',
  '5805517d-70c9-405c-94f1-5b1c30d0f5ac',
  'c0a808ed-2425-4f26-8fc5-74e18843a30b',
  '4a3375cd-ea4e-4d33-8b64-86afd699598c',
  'e1c30b4c-f3d6-4ffd-9f5a-cf51c90cce83',
  '7e94cb1e-c1c0-420e-92c5-

In [61]:
query = "What is Arduino?"
results = vector_store.similarity_search(query, k=2)
for result in results:
    print(result.page_content)

Arduino Basics and Applications


Arduino Basics and Applications:

Arduino is an open-source electronics platform based on easy-to-use hardware and software. It is popular for creating interactive projects.

Components of an Arduino:
1. **Microcontroller**: The "brain" of the Arduino board, typically the ATmega328 chip in many Arduino models.
2. **Digital and Analog Pins**: Used to interface with various electronic components, such as sensors and actuators.
3. **Power Supply**: Provides the necessary power for the board, either through USB or external power sources.

Programming Arduino:
1. **Arduino IDE**: The official integrated development environment used to write and upload code to the board.
2. **Sketch**: A program written for the Arduino, usually written in C++.
3. **Libraries**: Prewritten code that simplifies the control of specific components, such as motors or sensors.
Arduino Basics and Applications


Arduino Basics and Applications:

Arduino is an open-source electronics

# Retrievers

In [ ]:
retriever=vector_store.as_retriever(search_kwargs={"k":3},search_type="similarity")

In [63]:
retriever.invoke("What is ESP?")

[Document(id='c1a8ed75-6cea-404a-93a9-c66b1b16da2d', metadata={'source': 'e:\\Mentora\\Files\\EEE\\ESP32_ESP8266_Guide.docx'}, page_content='Programming Environment:\n1. **Arduino IDE**: You can use the same Arduino IDE to program the ESP32/ESP8266, with some additional board definitions.\n2. **ESP-IDF**: A more advanced framework for programming ESP32 devices, providing more control over hardware.\n\nExample Projects:\n1. **Smart Home System**: Using the ESP32, you can control lights and fans remotely through Wi-Fi using an app or a web interface.\n2. **Weather Station**: With a temperature and humidity sensor, the ESP32 can send data to the cloud, where it can be monitored in real-time.'),
 Document(id='84213f23-d70d-4ecf-940c-972d68dec4a7', metadata={'source': 'e:\\Mentora\\Files\\EEE\\ESP32_ESP8266_Guide.docx'}, page_content='Programming Environment:\n1. **Arduino IDE**: You can use the same Arduino IDE to program the ESP32/ESP8266, with some additional board definitions.\n2. **ESP

In [64]:
def format_docs(docs):
    final_text = []
    for doc in docs:
        # Clean up the text by removing excessive whitespace and line breaks
        cleaned_text = " ".join(doc.page_content.split())
        final_text.append(cleaned_text)
    return "\n\n".join(final_text)

# 4. Define the Message system

In [94]:
prompt=PromptTemplate(
    template='''
    You are given Two Inputs:
    1. Previous conversation
    2. Document Context
    3. Extra Documents provided by User

    Based on only these source, answer the query. If the answer is not present there, Search in User Provided Context. IF answer is still not found say: "I don't know"

    Previous conversation:
    {chat_history}
    
    Document context:
    {retrieved_docs}
    
    User Provided Context (This is the most important SOURCE for you If user provides any context by himself):
    {extra_docs}


    Current question: {query}
    
    Please answer the question using the document context and previous conversation only if the answer is present.
    ''',
    input_variables=["chat_history", "retrieved_docs", "extra_docs", "query"]
)

In [111]:
# Simple Chat History Management
chat_history = []

def add_to_history(human_msg, ai_msg):
    """Add conversation to history"""
    chat_history.append(f"Human: {human_msg}")
    chat_history.append(f"AI: {ai_msg}")
    
    # Save to file
    with open("chat_history.txt", "a", encoding="utf-8") as f:
        for msg in chat_history:
            f.write(msg + "\n")
    chat_history.clear()  # Clear history after saving to avoid duplication

def get_chat_history():
    with open("chat_history.txt", "r", encoding="utf-8") as f:
        chat_history = [line.strip() for line in f.readlines()]
    recent_history = chat_history[-10:]
    return "\n".join(recent_history)


def clear_chat_history():
    """Clear chat history"""
    global chat_history
    chat_history = []
    with open("chat_history.txt", "w") as f:
        f.write("")


In [ ]:
def load_user_file(path="Files/User"):
    folder_path=os.path.abspath(path)
    files=os.listdir(folder_path)
    files = [os.path.join(folder_path, file) for file in files if os.path.isfile(os.path.join(folder_path, file))]
    for file in files:
        content=document_loader(file)
            # Process the content as needed
    return content


files=load_user_file()
files

In [ ]:
parallel_chain= RunnableParallel(
    {
    'retrieved_docs': RunnableLambda(lambda x: x["query"]) | retriever | RunnableLambda(format_docs),
    'chat_history': RunnableLambda(lambda x: get_chat_history()),
    'query': RunnablePassthrough()
    }
)

main_chain = parallel_chain | prompt | llm | parser

In [105]:
def simple_chat(query):
    """Simple chat function with history"""
    # Get response
    response = main_chain.invoke({"query": query})
    
    # Add to history
    add_to_history(query, response)
    
    return response

# Test the chat
response = simple_chat("What is Network Layers?")
print(f"Response: {response}")

print(f"\nChat History:\n{get_chat_history()}")

Response: Hysteresis losses can be effectively reduced by the injection of small amounts of silicon into the magnetic core, constituting some 2% or 3% of the total composition of the core. This must be done carefully, however, because too much silicon makes the core brittle and difficult to machine into the shape desi red.

Chat History:
No previous conversation.


In [109]:
response = simple_chat("What amount of silicon should be injected?")
print(f"Response: {response}")

print(f"\nChat History:\n{get_chat_history()}")

Response: The document context states that small amounts of silicon (2% or 3%) should be injected into the magnetic core to effectively reduce hysteresis losses.

Chat History:
No previous conversation.


In [102]:
# Simple utility functions
def show_history():
    """Show current chat history"""
    print("=== CHAT HISTORY ===")
    print(get_chat_history())

def clear_history():
    """Clear chat history"""
    clear_chat_history()
    print("Chat history cleared!")

# Example usage:
show_history()
clear_history()

=== CHAT HISTORY ===
No previous conversation.
Chat history cleared!
